In [1]:
#!pip install pymongo

from pymongo import MongoClient
import pandas as pd
import numpy as np
client = MongoClient()
db = client.openfoodfacts
collection = db.products

import sys
sys.path.append('../')
from func import *

In [6]:
######################################################
#    1)  FEATURE SELECTION FROM OPEN FF              #
######################################################

In [2]:
proj = { 
    'brands' : 1,
    'brands_tags' : 1,
    'product_name_fr' : 1,
    'product_name' : 1,
    'interface_version_modified' : 1,
    'ingredients_text' : 1, 
    'ingredients_n' : 1,
    'unknown_ingredients' : 1,
    'serving_size' : 1,
    'serving_quantity' : 1,
    'countries_hierarchy' : 1,
    'manufacturing_places' : 1,
    'cities_tags' : 1,
    'category_properties' : 1,
    'categories' : 1,
    'pnns_groups_1' : 1,
    'pnns_groups_2' : 1,
    'nutriments' : 1,
    'compared_to_category' : 1,
    'ingredients' : 1, 
    'salt_content' : 1, 
    'oil_content' : 1,
    'purchase_places' : 1,
    'countries_tags' : 1    
}

In [3]:
addFields = { 
    
    'ingredients' : "$ingredients.text",
    
    'salt_content' : {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ 
                   { '$eq' : [ "$$item.text", 'Sel' ] }, 
                   { '$eq' : [ "$$item.text", 'sel' ] },
                   { '$eq' : [ "$$item.text", 'SEL' ] },
                   { '$eq' : [ "$$item.text", 'VINAGE SEL' ] },
                   { '$eq' : [ "$$item.text", 'sel marin' ] },
                   { '$eq' : [ "$$item.text", 'Sel marin' ] },
                   { '$eq' : [ "$$item.text", 'salt' ] },
                   { '$eq' : [ "$$item.text", 'Salt' ] }] }
            }
    },
    
    
    'oil_content': {
            '$filter' : {
               'input' : "$ingredients",
               'as' : "item",
               'cond' : { '$or' : [ { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile extra vierge d\'olive' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive extra-vierge' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile d\'olive extra vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile dolive* vierge exta' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge' ] }, {
                   '$eq' : [ "$$item.text", 'huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive' ] }, {
                   '$eq' : [ "$$item.text", 'huile de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'huile d\'olive vierge extra extraite à froid' ] }, { 
                   '$eq' : [ "$$item.text", 'huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de tournesol' ] }, {
                   '$eq' : [ "$$item.text", 'huile de tournesol désodorisée' ] }, {
                   '$eq' : [ "$$item.text", 'Extra Virgin Olive Oil' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile de palme' ] }, {
                   '$eq' : [ "$$item.text", 'natives Olivenöl extra' ] }, { 
                   '$eq' : [ "$$item.text", 'Sonnenblumenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Olivenöl' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile Végétale' ] }, {
                   '$eq' : [ "$$item.text", 'Huile de colza' ] }, {
                   '$eq' : [ "$$item.text", 'huile végétale de colza' ] }, { 
                   '$eq' : [ "$$item.text", 'Huile végétale de colza' ] }, {
                   '$eq' : [ "$$item.text", 'Huile d\'olive vierge extra' ] } ] 
                    }
            }
    },
}

In [4]:
#EXTRACT OF FEATURES FROM MONGO DB "OPENFF"
#Faire une liste des labels puis boucle for. To do

def process_O_PRODUCT(data) :
    
    max_salt_content = []
    for l in data['salt_content'] :
        if l != None and len(l) > 0 and 'percent_max' in l[0]  :
            max_salt_content.append(l[0]['percent_max'])
        else :
            max_salt_content.append(0)

    data['max_salt_content'] = max_salt_content

    min_salt_content = []
    for l in data['salt_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            min_salt_content.append(l[0]['percent_min'])
        else :
            min_salt_content.append(0)

    data['min_salt_content'] = min_salt_content

    max_oil_content = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_max' in l[0]  :
            max_oil_content.append(l[0]['percent_max'])
        else :
            max_oil_content.append(0)

    data['max_oil_content'] = max_oil_content

    min_oil_content = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            min_oil_content.append(l[0]['percent_min'])
        else :
            min_oil_content.append(0)

    data['min_oil_content'] = min_oil_content

    oil_type = []
    for l in data['oil_content'] :
        if l != None and len(l) > 0 and 'percent_min' in l[0]  :
            oil_type.append(l[0]['text'])
        else :
            oil_type.append("")

    data['oil_type'] = oil_type


    ##################################################################
    ######    extraction of _100g                              #######
    ##################################################################

    energy_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'energy_100g' in l :
            energy_100g.append(l['energy_100g'])
        else :
            energy_100g.append(None)

    data['energy_100g'] = energy_100g


    energy_kcal_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'energy-kcal_100g' in l :
            energy_kcal_100g.append(l['energy-kcal_100g'])
        else :
            energy_kcal_100g.append(None)

    data['energy_kcal_100g'] = energy_kcal_100g


    saturated_fat_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'saturated-fat_100g' in l :
            saturated_fat_100g.append(l['saturated-fat_100g'])
        else :
            saturated_fat_100g.append(None)

    data['saturated_fat_100g'] = saturated_fat_100g


    fat_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fat_100g' in l :
            fat_100g.append(l['fat_100g'])
        else :
            fat_100g.append(None)

    data['fat_100g'] = fat_100g


    carbohydrates_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'carbohydrates_100g' in l :
            carbohydrates_100g.append(l['carbohydrates_100g'])
        else :
            carbohydrates_100g.append(None)

    data['carbohydrates_100g'] = carbohydrates_100g


    sugars_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'sugars_100g' in l :
            sugars_100g.append(l['sugars_100g'])
        else :
            sugars_100g.append(None)

    data['sugars_100g'] = sugars_100g


    fruits_vegetables_nuts_estimate_from_ingredients_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fruits-vegetables-nuts-estimate-from-ingredients_100g' in l :
            fruits_vegetables_nuts_estimate_from_ingredients_100g.append(l['fruits-vegetables-nuts-estimate-from-ingredients_100g'])
        else :
            fruits_vegetables_nuts_estimate_from_ingredients_100g.append(None)

    data['fruits_vegetables_nuts_estimate_from_ingredients_100g'] = fruits_vegetables_nuts_estimate_from_ingredients_100g


    proteins_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'proteins_100g' in l :
            proteins_100g.append(l['proteins_100g'])
        else :
            proteins_100g.append(None)

    data['proteins_100g'] = proteins_100g


    sodium_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'sodium_100g' in l :
            sodium_100g.append(l['sodium_100g'])
        else :
            sodium_100g.append(None)

    data['sodium_100g'] = sodium_100g



    salt_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'salt_100g' in l :
            salt_100g.append(l['salt_100g'])
        else :
            salt_100g.append(None)

    data['salt_100g'] = salt_100g


    nutrition_score_fr_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'nutrition-score-fr_100g' in l :
            nutrition_score_fr_100g.append(l['nutrition-score-fr_100g'])
        else :
            nutrition_score_fr_100g.append(None)

    data['nutrition_score_fr_100g'] = nutrition_score_fr_100g


    fiber_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'fiber_100g' in l :
            fiber_100g.append(l['fiber_100g'])
        else :
            fiber_100g.append(None)

    data['fiber_100g'] = fiber_100g


    nova_group_100g = []
    for l in data['nutriments'] :
        if len(l) > 0 and 'nova-group_100g' in l :
            nova_group_100g.append(l['nova-group_100g'])
        else :
            nova_group_100g.append(None)

    data['nova_group_100g'] = nova_group_100g



In [10]:
######################################################
#    2)  EXTRACT OF OPEN FF FOR N_EAN13 (NIELSEN DB) #
######################################################

In [5]:
#importing EAN13 present in NIELSEN DB
EAN13_temp = pd.read_csv('../N_NIELSEN/output_data/N_EAN13_U.csv', index_col=0, names=['EAN13']).values.flatten()

EAN13 = []
for ean in EAN13_temp :
    EAN13.append(str(ean))
    
N_EAN13 = EAN13[1:]
len(N_EAN13)

1425

In [6]:
#Extracting N_EAN13 from O_products
match = { "_id" : { "$in" : N_EAN13 } }


c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields },
])

O_data_EAN13 = pd.DataFrame(list(c_f))
O_data_EAN13.rename(columns={'_id': 'EAN13'},inplace=True)
O_data_EAN13['brands_tags_str'] = O_data_EAN13['brands_tags'].astype(str)
O_data_EAN13.shape

(1100, 25)

In [7]:
# Add two columns based on ciqual_food_name fr and en
import math

O_data_EAN13['ciqual_food_name:fr'] = ''
O_data_EAN13['ciqual_food_name:en'] = ''

temp_cfn_fr = []
temp_cfn_en = []

for tc in O_data_EAN13['category_properties'] :
    
    if isinstance(tc, dict) :
        
        if 'ciqual_food_name:fr' in tc :
            temp_cfn_fr.append(tc['ciqual_food_name:fr'])
        else :
            temp_cfn_fr.append('')
        
        if 'ciqual_food_name:en' in tc :
            temp_cfn_en.append(tc['ciqual_food_name:en'])
        else :
            temp_cfn_en.append('')
    
    else :
        temp_cfn_fr.append('')
        temp_cfn_en.append('')
        
O_data_EAN13['ciqual_food_name:fr'] = temp_cfn_fr
O_data_EAN13['ciqual_food_name:en'] = temp_cfn_en

In [8]:
#EXTRACTING ADDITIONAL FEATURES FROM OPEN FF (NUTRIMENTS)
process_O_PRODUCT(O_data_EAN13)

In [9]:
#Reordering columns
O_data_EAN13 = O_data_EAN13[[
    'EAN13', 
    'product_name_fr', 'product_name', 
    'brands', 'brands_tags_str', 'brands_tags', 
    'serving_size', 'serving_quantity',
    'countries_hierarchy', 'cities_tags', 'manufacturing_places', 
    'purchase_places', 'countries_tags',
    'categories', 'category_properties', 'pnns_groups_1', 'pnns_groups_2',
    'compared_to_category',
        
    'interface_version_modified',
    
    'ingredients_n',
    'ingredients_text',
    'ingredients', 

    'salt_content', 'max_salt_content', 'min_salt_content',
    'oil_content', 'max_oil_content', 'min_oil_content', 'oil_type',
    
    'nutriments', 
    
    'energy_100g', 'energy_kcal_100g', 'saturated_fat_100g', 'fat_100g',
    'carbohydrates_100g', 'sugars_100g',
    'fruits_vegetables_nuts_estimate_from_ingredients_100g',
    'proteins_100g', 'sodium_100g', 'salt_100g',
    'nutrition_score_fr_100g', 'fiber_100g', 'nova_group_100g']]

In [15]:
#Saving finalized OpenFF export
#O_data_EAN13.to_csv('output_data/O_produits_EAN13.csv')
O_data_EAN13.to_excel('output_data/O_produits_EAN13.xls')

In [ ]:
##########################################################
#   O_data_EAN13 exploration

In [11]:
O_data_EAN13.shape

(1100, 43)

In [12]:
O_data_EAN13.head()

,EAN13,product_name_fr,product_name,brands,brands_tags_str,brands_tags,serving_size,serving_quantity,countries_hierarchy,cities_tags,...,fat_100g,carbohydrates_100g,sugars_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,proteins_100g,sodium_100g,salt_100g,nutrition_score_fr_100g,fiber_100g,nova_group_100g
0,3017239003969,Sauce aux Olives Provençale,Sauce aux Olives Provençale,Tramier,['tramier'],[tramier],NaN,NaN,[en:france],[],...,11.0,3.1,2.7,53.166667,1.3,0.84,2.10,10.0,NaN,3.0
1,3017239003976,Sauce aux olives façon pesto vert,Sauce aux olives façon pesto vert,Tramier,['tramier'],[tramier],NaN,NaN,[en:france],[],...,25.0,5.8,2.6,0.000000,1.7,0.30,0.75,8.0,NaN,4.0
2,3017239003983,Sauce aux olives façon pesto rosso,Sauce aux olives façon pesto rosso,Tramier,['tramier'],[tramier],190g,190,[en:france],[],...,20.0,6.3,4.3,46.000000,2.8,0.48,1.20,7.0,NaN,4.0
3,3017239003990,NaN,Sauce aux Olives Citron Confit,Tramier,['tramier'],[tramier],NaN,NaN,[en:france],NaN,...,7.5,4.2,3.5,NaN,1.4,0.56,1.40,NaN,NaN,NaN
4,3017239004003,Sauce aux olives champignons,Sauce aux olives champignons,Tramier,['tramier'],[tramier],NaN,NaN,[en:france],[],...,7.0,4.1,2.5,36.000000,1.5,0.48,1.20,7.0,NaN,4.0


In [14]:
O_data_EAN13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 43 columns):
EAN13                                                    1100 non-null object
product_name_fr                                          1073 non-null object
product_name                                             1093 non-null object
brands                                                   912 non-null object
brands_tags_str                                          1100 non-null object
brands_tags                                              912 non-null object
serving_size                                             92 non-null object
serving_quantity                                         92 non-null object
countries_hierarchy                                      1100 non-null object
cities_tags                                              588 non-null object
manufacturing_places                                     564 non-null object
purchase_places                            

In [15]:
O_data_EAN13.keys()

Index(['EAN13', 'product_name_fr', 'product_name', 'brands', 'brands_tags_str',
       'brands_tags', 'serving_size', 'serving_quantity',
       'countries_hierarchy', 'cities_tags', 'manufacturing_places',
       'purchase_places', 'countries_tags', 'categories',
       'category_properties', 'pnns_groups_1', 'pnns_groups_2',
       'compared_to_category', 'interface_version_modified', 'ingredients_n',
       'ingredients_text', 'ingredients', 'salt_content', 'max_salt_content',
       'min_salt_content', 'oil_content', 'max_oil_content', 'min_oil_content',
       'oil_type', 'nutriments', 'energy_100g', 'energy_kcal_100g',
       'saturated_fat_100g', 'fat_100g', 'carbohydrates_100g', 'sugars_100g',
       'fruits_vegetables_nuts_estimate_from_ingredients_100g',
       'proteins_100g', 'sodium_100g', 'salt_100g', 'nutrition_score_fr_100g',
       'fiber_100g', 'nova_group_100g'],
      dtype='object')

In [ ]:
######################################################
#    3)  OPEN FF CATEGORY ANALYSIS.                  #
######################################################

In [16]:
#CHECKOUT FOR SPECIFIC VALUES IN A COLUMN

    

In [16]:
temp = O_data_EAN13[['EAN13', 'product_name', 'product_name_fr', 'categories', 'category_properties'
       , 'pnns_groups_1', 'pnns_groups_2', 'compared_to_category',
       'serving_quantity', 'serving_size', 'ingredients_text']]

In [17]:
temp.head(20)

,EAN13,product_name,product_name_fr,categories,category_properties,pnns_groups_1,pnns_groups_2,compared_to_category,serving_quantity,serving_size,ingredients_text
0,3017239003969,Sauce aux Olives Provençale,Sauce aux Olives Provençale,"Epicerie, Sauces, Sauces aux légumes, Sauces a...",{},Fat and sauces,Dressings and sauces,en:provencale-sauces,NaN,NaN,"Tomate (51%), olive noire (36%), huile d'olive..."
1,3017239003976,Sauce aux olives façon pesto vert,Sauce aux olives façon pesto vert,"Epicerie, Sauces, Sauces aux légumes, Sauces a...",{},Fat and sauces,Dressings and sauces,en:vegetable-sauces,NaN,NaN,"olive verte égouttée (33%), huile de tournesol..."
2,3017239003983,Sauce aux olives façon pesto rosso,Sauce aux olives façon pesto rosso,"Epicerie, Sauces, Sauces Pesto, Sauces Pesto R...","{'ciqual_food_name:fr': 'Sauce pesto rosso, pr...",Fat and sauces,Dressings and sauces,en:red-pestos,190,190g,"Tomate 46%, olive 18% (dont olive verte égoutt..."
3,3017239003990,Sauce aux Olives Citron Confit,NaN,NaN,{},unknown,unknown,NaN,NaN,NaN,NaN
4,3017239004003,Sauce aux olives champignons,Sauce aux olives champignons,"Epicerie, Sauces, Sauces tomate, Sauces tomate...","{'ciqual_food_name:en': 'Tomato sauce, w mushr...",Fat and sauces,Dressings and sauces,fr:sauces-tomates-aux-champignons,NaN,NaN,"Tomate (36 %), champignon (24 %), olive verte ..."
5,3017239004010,Tramier,Tramier,"Epicerie, Sauces, Sauces aux légumes, Sauces a...",{},Fat and sauces,Dressings and sauces,en:vegetable-sauces,NaN,NaN,"Tomate (50 %), olive (27 %) (dont olive verte ..."
6,3021690017243,bolognaise boeuf rissolé,bolognaise boeuf rissolé,NaN,{},unknown,unknown,NaN,NaN,NaN,NaN
7,3021690017250,Napolitaine,Napolitaine,"Epicerie, Sauces, Sauces tomate, Sauces napoli...",{},Fat and sauces,Dressings and sauces,en:neapolitan-sauces,NaN,NaN,pulpe de tomate avec morceaux et purée de toma...
8,3021690017267,La sauce italienne à la viande rôtie,La sauce italienne à la viande rôtie,"Epicerie, Produits à la viande, Sauces, Sauces...",{},Fat and sauces,Dressings and sauces,en:tomato-sauces,95,95 g,Purée de tomate mi-réduite à base de concentré...
9,3021690017274,Sauce tomate de Provence Languedoc,Sauce tomate de Provence Languedoc,"Epicerie, Sauces, Sauces tomate, Sauces proven...",{},Fat and sauces,Dressings and sauces,en:provencale-sauces,95,95g,purée de tomate de Provence et Languedoc à bas...


In [ ]:
#categories found in O_OPENFF database to check
# 'categories', 
# 'category_properties'
#'pnns_groups_1', 
#'pnns_groups_2',
#'compared_to_category'


In [ ]:
###################################################
# 1) column : categories

In [18]:
cat = values_U_not_nan(temp, 'categories')
print(len(cat)-1)
cat[1:]

173


["Aliments et boissons à base de végétaux, Aliments d'origine végétale, Epicerie, Conserves, Aliments à base de fruits et de légumes, Aliments à base de plantes en conserve, Légumes et dérivés, Légumes en conserve, Tomates et dérivés, Sauces, Tomates, Tomates en conserve, Sauces tomate",
 "Aliments et boissons à base de végétaux, Aliments d'origine végétale, Petit-déjeuners, Produits à tartiner, Pâtes à tartiner végétales, Produits à tartiner sucrés, en:fruit-preserves",
 "Aliments et boissons à base de végétaux, Aliments d'origine végétale, Produits à tartiner, Produits à tartiner salés, Pâtes à tartiner végétales, Houmous",
 'Boissons',
 'Comestibles, Salsas',
 'Comestibles, Salsas, Salsas de tomate',
 'Comestibles, Salsas, Salsas de tomate, Salsas de tomate frito',
 'Conserves, Viandes, Viandes en conserve',
 'Dairies,Fermented foods,Fermented milk products,Cheeses,Italian cheeses,Parmigiano-Reggiano,Pasta sauces',
 'Epicerie, Condiments',
 'Epicerie, Condiments, Assaisonnements, Ma

In [41]:
selection_O_CAT_categories = ["Aliments et boissons à base de végétaux, Aliments d'origine végétale, Epicerie, Conserves, Aliments à base de fruits et de légumes, Aliments à base de plantes en conserve, Légumes et dérivés, Légumes en conserve, Tomates et dérivés, Sauces, Tomates, Tomates en conserve, Sauces tomate",
# "Aliments et boissons à base de végétaux, Aliments d'origine végétale, Petit-déjeuners, Produits à tartiner, Pâtes à tartiner végétales, Produits à tartiner sucrés, en:fruit-preserves",
 "Aliments et boissons à base de végétaux, Aliments d'origine végétale, Produits à tartiner, Produits à tartiner salés, Pâtes à tartiner végétales, Houmous",
# 'Boissons',
 'Comestibles, Salsas',
 'Comestibles, Salsas, Salsas de tomate',
 'Comestibles, Salsas, Salsas de tomate, Salsas de tomate frito',
 #'Conserves, Viandes, Viandes en conserve',
 'Dairies,Fermented foods,Fermented milk products,Cheeses,Italian cheeses,Parmigiano-Reggiano,Pasta sauces',
 'Epicerie, Condiments',
 'Epicerie, Condiments, Assaisonnements, Marinades',
 'Epicerie, Condiments, Sauces, Sauces tomate',
 'Epicerie, Conserves, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes',
 'Epicerie, Conserves, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Sauces bolognaises',
 'Epicerie, Conserves, Sauces, Sauces aux champignons, Sauce pour Risotto aux Champignons, Sauces pour risotto',
 'Epicerie, Conserves, Sauces, Sauces pour pâtes, Sauces tomate, Sauces Arrabiata, Sauces tomates pimentées',
 'Epicerie, Conserves, Sauces, Sauces tomate, Sauces provençales',
 'Epicerie, Plats préparés, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Sauces tomate, Sauces bolognaises',
 'Epicerie, Produits à la viande, Sauces, Sauces tomate, Sauces à la viande, Sauces pour pâtes',
 'Epicerie, Produits à la viande, Sauces, Sauces tomate, Sauces à la viande, Sauces pour pâtes, Sauces bolognaises',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Produits sans gluten, Sauces bolognaises',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Sauces bolognaises',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Sauces bolognaises, Sauce préparée',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces pour pâtes, Sauces tomate, Sauces aux légumes, Sauces bolognaises, Sauces tomates aux légumes',
 'Epicerie, Produits à la viande, Sauces, Sauces à la viande, Sauces tomate',
 'Epicerie, Produits à tartiner, Sauces, Condiments à tartiner',
 'Epicerie, Sauces',
 'Epicerie, Sauces, Harissas',
 'Epicerie, Sauces, Pistou',
 'Epicerie, Sauces, Sauce Bolognese',
 'Epicerie, Sauces, Sauce aux poivrons et aubergines',
 'Epicerie, Sauces, Sauce à la crème, Sauces aux champignons',
 'Epicerie, Sauces, Sauces Pesto',
 "Epicerie, Sauces, Sauces Pesto, Pesto d'artichauts",
 'Epicerie, Sauces, Sauces Pesto, Pestos au basilic',
 'Epicerie, Sauces, Sauces Pesto, Sauces Pesto Rosso',
 'Epicerie, Sauces, Sauces Pesto, Sauces tomate, Sauces Pesto Rosso, Sauces tomates au basilic',
 'Epicerie, Sauces, Sauces Pesto, it:Pesto alla calabrese',
 'Epicerie, Sauces, Sauces Teriyaki',
 'Epicerie, Sauces, Sauces au curry',
 'Epicerie, Sauces, Sauces au curry, Sauces Korma',
 'Epicerie, Sauces, Sauces au fromage',
 'Epicerie, Sauces, Sauces au fromage, Sauces pour risottos',
 'Epicerie, Sauces, Sauces au lait de coco',
 'Epicerie, Sauces, Sauces au poivre',
 'Epicerie, Sauces, Sauces aux champignons',
 'Epicerie, Sauces, Sauces aux champignons, Sauce pour Risotto',
 'Epicerie, Sauces, Sauces aux légumes',
 'Epicerie, Sauces, Sauces aux légumes, Sauces aux asperges',
 'Epicerie, Sauces, Sauces aux légumes, Sauces aux olives',
 'Epicerie, Sauces, Sauces aux légumes, Sauces aux olives, Sauces provençales',
 'Epicerie, Sauces, Sauces barbecue',
 'Epicerie, Sauces, Sauces barbecue, Sauces Teriyaki',
 'Epicerie, Sauces, Sauces basquaises',
 'Epicerie, Sauces, Sauces béchamel',
 'Epicerie, Sauces, Sauces carbonara',
 'Epicerie, Sauces, Sauces chili',
 'Epicerie, Sauces, Sauces indiennes, Sauces tikka masala',
 'Epicerie, Sauces, Sauces pimentées',
 'Epicerie, Sauces, Sauces pour pizzas',
 'Epicerie, Sauces, Sauces pour poissons',
 'Epicerie, Sauces, Sauces pour poissons, Sauces armoricaines',
 'Epicerie, Sauces, Sauces pour poissons, Sauces au beurre blanc',
 'Epicerie, Sauces, Sauces pour pâtes',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto',
 "Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Pesto d'artichauts",
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Pestos au basilic',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Produits sans gluten',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Sauce Pesto Doux tomates',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Sauces Pesto Rosso',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Sauces chaudes',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces Pesto, Sauces tomate, Sauces Pesto Rosso',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces au fromage',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces carbonara',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces provençales',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, Sauces Arrabiata',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, Sauces Arrabiata, Sauces pimentées',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, Sauces Arrabiata, Sauces tomates pimentées',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, Sauces tomates au basilic',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, Sauces tomates pimentées',
 'Epicerie, Sauces, Sauces pour pâtes, Sauces tomate, en:food',
 'Epicerie, Sauces, Sauces pour pâtes, en:Puttanesca sauces',
 'Epicerie, Sauces, Sauces provençales',
 'Epicerie, Sauces, Sauces provençales, Sauces tomates à la provençale',
 'Epicerie, Sauces, Sauces tomate',
 'Epicerie, Sauces, Sauces tomate, Ketchup',
 'Epicerie, Sauces, Sauces tomate, Sauce tomate cuisinée aux légumes',
 'Epicerie, Sauces, Sauces tomate, Sauce tomates cuisinées',
 'Epicerie, Sauces, Sauces tomate, Sauce tomates frites',
 'Epicerie, Sauces, Sauces tomate, Sauces aux champignons',
 'Epicerie, Sauces, Sauces tomate, Sauces aux légumes, Sauces tomates au basilic, Sauces tomates aux légumes',
 'Epicerie, Sauces, Sauces tomate, Sauces aux légumes, Sauces tomates aux légumes',
 'Epicerie, Sauces, Sauces tomate, Sauces napolitaines',
 'Epicerie, Sauces, Sauces tomate, Sauces napolitaines, en:Food',
 'Epicerie, Sauces, Sauces tomate, Sauces pour lasagnes',
 'Epicerie, Sauces, Sauces tomate, Sauces pour pâtes',
 'Epicerie, Sauces, Sauces tomate, Sauces pour pâtes, Sauces Arrabiata',
 'Epicerie, Sauces, Sauces tomate, Sauces pour pâtes, Sauces Arrabiata, Sauces tomates pimentées',
 'Epicerie, Sauces, Sauces tomate, Sauces pour pâtes, Sauces napolitaines',
 'Epicerie, Sauces, Sauces tomate, Sauces pour pâtes, Sauces provençales',
 'Epicerie, Sauces, Sauces tomate, Sauces provençales',
 'Epicerie, Sauces, Sauces tomate, Sauces tomate au mascarpone',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates au basilic',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates au basilic, Sauce páte',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates au basilic, Sauces tomates aux olives',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates au basilic, en:food',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates au parmesan',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates aux champignons',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates aux olives',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates aux olives, en:Puttanesca sauces, Sauces puttanesca',
 'Epicerie, Sauces, Sauces tomate, Sauces tomates pimentées',
 'Epicerie, Sauces, Sauces tomate, Souces pour pâtes',
 'Epicerie, Sauces, Sauces tomate, de:Tomatensauce',
 'Epicerie, Sauces, Sauces tomate, en:Sauce tomate cuisinée',
 'Epicerie, Sauces, Sauces tomate, en:food',
 'Epicerie, Sauces, Sauces vegan',
 'Epicerie,Sauces,Sauces Teriyaki',
 'Epicerie,Sauces,Sauces tomate',
 'Groceries, Meat-based products, Sauces, Meat-based sauces, Pasta sauces, Bolognese sauces',
 'Groceries, Sauces',
 'Groceries, Sauces, Hot sauces',
 'Groceries, Sauces, Pasta sauces, Pestos, Green pestos',
 'Groceries, Sauces, Pasta sauces, Pestos, Red pestos',
 'Groceries, Sauces, Pasta sauces, Tomato sauces, Arrabbiata sauces',
 'Groceries, Sauces, Pestos, Green pestos',
 'Groceries, Sauces, Puttanesca sauces',
 'Groceries, Sauces, Tomato sauces, Pasta sauces, Arrabbiata sauces',
 'Groceries, Sauces, Tomato sauces, Vegetable sauces, fr:Sauces tomates aux légumes',
 'Groceries, Sauces, en:green-pestos',
 'Groceries,Sauces,Pestos,Green pestos',
 #'Lebensmittel, Fertiggerichte',
 #'Lebensmittel, Saucen, Nudelsoßen',
 #'Lebensmittel, Saucen, Nudelsoßen, Pestos',
 #'Lebensmittel, Saucen, Nudelsoßen, Pestos, Pesto alla Genovese',
 #'Lebensmittel, Saucen, Nudelsoßen, Pestos, Rote Pestos',
 #'Lebensmittel, Saucen, Nudelsoßen, Pestos, Tomatensaucen, Rote Pestos, en:food',
 #'Lebensmittel, Saucen, Nudelsoßen, Tomatensaucen, Arrabiata-Soßen',
 #'Lebensmittel, Saucen, Nudelsoßen, Tomatensaucen, fr:Sauces tomates aux olives, en:Food',
 #'Lebensmittel, Saucen, Pestos',
 #'Lebensmittel, Saucen, Tomatensaucen',
 #'Matières grasses',
 #'Pflanzliche Lebensmittel und Getränke, Pflanzliche Lebensmittel, Frucht- und gemüsebasierte Lebensmittel, Gemüsebasierte Lebensmittel, Tomaten und Tomatenprodukte',
 #'Plant-based foods and beverages, Plant-based foods, Fruits and vegetables based foods, Vegetables based foods, Tomatoes and their products, Tomato pastes',
 #'Plant-based foods and beverages, Plant-based foods, Fruits and vegetables based foods, Vegetables based foods, Tomatoes and their products, Tomatoes',
 #'Plant-based foods and beverages, Plant-based foods, Groceries, Fruits and vegetables based foods, Vegetables based foods, Sauces, Tomatoes and their products, Tomatoes, Tomato sauces',
 #'Plantaardige levensmiddelen en dranken, Plantaardige levensmiddelen, Voedsel op basis van fruit en groenten, Kant-en-klaar maaltijden, Soepen, Groentesoepen, Koude soepen, Gazpachos',
 #'Plats préparés, Produits déshydratés, Produits lyophilisés à reconstituer, Soupes, Plats préparés déshydratés, Soupes déshydratées, Potage-deshydrate',
 #'Plats préparés, Taboulés',
 #'Produits à la viande, Sauces à la viande',
 'Produits à tartiner, Condiments à tartiner',
 #'Ragu',
 'Sauce apéritive',
 'Sauce assaisonnement',
 'Sauce aux cèpes',
 'Sauce tomate cuisinée',
 "Sauce tomate à l'arrabbiata",
 'Sauce wok',
 'Sauces chaudes',
 'Souces pour les pâtes',
 'Sughi', #jus en italien
 'Sugo', #jus 'pluriel en italien
 #'de:Tomatensauce',
 'en:Groceries, Alimenti in scatola, Salse, Sughi per pasta, Salse di pomodoro, en:Pizza sauces, fr:Sauces tomates au basilic',
 'en:Groceries, Condimenti',
 'en:Groceries, Salse, Pesto, Pesto alla Genovese',
 'en:Groceries, Salse, Salse di pomodoro',
 'en:Groceries, en:Sauces',
 'en:Puttanesca sauces',
 'en:groceries,Salse,Sughi per pasta,Pesto',
 'en:pestos',
 'it:pesto-alla-calabrese']

In [42]:
nb_prod_cat_empty = temp['categories'].isna().sum()
print( 'number of products without this field: ', nb_prod_cat_empty, '(', round(nb_prod_cat_empty/temp.shape[0]*100, 2), '%)')
print('number of different categories: ', nb_prod_cat_empty)

number of products without this field:  279 ( 25.36 %)
number of different categories:  279


In [ ]:
###################################################
# 2) column : category_properties

In [46]:
temp_cat = temp['category_properties']
temp_cat = temp_cat[(temp_cat != {}) & (~temp_cat.isna())]

ciqual_food_names_fr = []
ciqual_food_names_en = []

for tc in temp_cat :
    if 'ciqual_food_name:fr' in tc :
        ciqual_food_names_fr.append(tc['ciqual_food_name:fr'])
    if 'ciqual_food_name:en' in tc :
        ciqual_food_names_en.append(tc['ciqual_food_name:en'])

ciqual_food_names_fr_U = set(ciqual_food_names_fr)
ciqual_food_names_en_U = set(ciqual_food_names_en)

In [44]:
nb_prod_cat_empty = temp['category_properties'].isna().sum()
nb_prod_cat_empty += (temp['category_properties'] == {}).sum()
print( 'number of products without this field: ', nb_prod_cat_empty, '(', round(nb_prod_cat_empty/temp.shape[0]*100, 2), '%)')
print('number of different categories: ', nb_prod_cat_empty)

number of products without this field:  841 ( 76.45 %)
number of different categories:  841


In [45]:
print( 'ciqual_food_names_fr length : ', len(ciqual_food_names_fr))
ciqual_food_names_fr

ciqual_food_names_fr length :  23


{'Fromage -aliment moyen-',
 'Harissa -sauce condimentaire-',
 'Houmous',
 'Ketchup',
 'Sauce armoricaine, préemballée',
 'Sauce au beurre blanc, préemballée',
 'Sauce au curry, préemballée',
 'Sauce aux champignons, préemballée',
 'Sauce barbecue, préemballée',
 'Sauce basquaise ou Sauce aux poivrons, préemballée',
 'Sauce béchamel, préemballée',
 'Sauce carbonara, préemballée',
 'Sauce indienne type tandoori ou tikka masala, préemballée',
 'Sauce pesto rosso, préemballée',
 'Sauce pesto, préemballée',
 'Sauce teriyaki, préemballée',
 'Sauce tomate aux champignons, préemballée',
 'Sauce tomate aux olives, préemballée',
 'Sauce tomate à la viande ou Sauce bolognaise, préemballée',
 'Soupe -aliment moyen-',
 'Soupe froide type Gaspacho ou Gazpacho',
 'Taboulé ou Salade de couscous, préemballé',
 'Tomate, concentré, appertisé'}

In [24]:
#Selection 
selection_ciqual_food_names_fr = [
 #'Fromage -aliment moyen-',
 'Harissa -sauce condimentaire-',
 #'Houmous',
 'Ketchup',
 'Sauce armoricaine, préemballée',
 'Sauce au beurre blanc, préemballée',
 'Sauce au curry, préemballée',
 'Sauce aux champignons, préemballée',
 'Sauce barbecue, préemballée',
 'Sauce basquaise ou Sauce aux poivrons, préemballée',
 'Sauce béchamel, préemballée',
 'Sauce carbonara, préemballée',
 'Sauce indienne type tandoori ou tikka masala, préemballée',
 'Sauce pesto rosso, préemballée',
 'Sauce pesto, préemballée',
 'Sauce teriyaki, préemballée',
 'Sauce tomate aux champignons, préemballée',
 'Sauce tomate aux olives, préemballée',
 'Sauce tomate à la viande ou Sauce bolognaise, préemballée',
 #'Soupe -aliment moyen-',
 #'Soupe froide type Gaspacho ou Gazpacho',
 #'Taboulé ou Salade de couscous, préemballé',
 'Tomate, concentré, appertisé']

In [25]:
print( 'ciqual_food_names_en length : ', len(ciqual_food_names_en))
ciqual_food_names_en

ciqual_food_names_en length :  25


{'Armorican-style sauce, prepacked',
 'Barbecue sauce, prepacked',
 'Basque-style sauce or tomato sauce with sweet peppers, prepacked',
 'Bechamel sauce, prepacked',
 'Carbonara sauce -cream sauce with lardoons-, prepacked',
 'Cheese sauce for risotto or pasta, prepacked',
 'Curry sauce, prepacked',
 'Harissa -hot spicy sauce-, prepacked',
 'Hummus',
 'Indian-style sauce, tandoori or garam masala type, prepacked',
 'Ketchup',
 'Mushroom sauce, prepacked',
 "Parmesan cheese, from cow's milk",
 'Sauce, pepper, prepacked',
 'Sauce, pesto rosso, prepacked',
 'Sauce, pesto, prepacked',
 'Soup -average-',
 'Soup, gazpacho, cold',
 'Tabbouleh, prepacked',
 'Teriyaki sauce, prepacked',
 'Tomato paste, concentrated, canned',
 'Tomato sauce, w mushrooms, prepacked',
 'Tomato sauce, w olives, prepacked',
 'Tomato sauce, with meat or Bolognese sauce, prepacked',
 'White butter sauce, prepacked'}

In [26]:
selection_ciqual_food_names_en = [
 'Armorican-style sauce, prepacked',
 'Barbecue sauce, prepacked',
 'Basque-style sauce or tomato sauce with sweet peppers, prepacked',
 'Bechamel sauce, prepacked',
 'Carbonara sauce -cream sauce with lardoons-, prepacked',
 'Cheese sauce for risotto or pasta, prepacked',
 'Curry sauce, prepacked',
 'Harissa -hot spicy sauce-, prepacked',
 'Hummus',
 'Indian-style sauce, tandoori or garam masala type, prepacked',
 'Ketchup',
 'Mushroom sauce, prepacked',
 #"Parmesan cheese, from cow's milk",
 'Sauce, pepper, prepacked',
 'Sauce, pesto rosso, prepacked',
 'Sauce, pesto, prepacked',
 #'Soup -average-',
 #'Soup, gazpacho, cold',
 #'Tabbouleh, prepacked',
 'Teriyaki sauce, prepacked',
 'Tomato paste, concentrated, canned',
 'Tomato sauce, w mushrooms, prepacked',
 'Tomato sauce, w olives, prepacked',
 'Tomato sauce, with meat or Bolognese sauce, prepacked',
 'White butter sauce, prepacked']

In [ ]:
###################################################
# column : pnns_groups_1

In [27]:
cat_prop = values_U_not_nan(temp, 'pnns_groups_1')
nb_prod_cat_empty = df_with_text(temp, 'pnns_groups_1', 'unknown').shape[0]
print( 'number of products without this field: ', nb_prod_cat_empty, '(', round(nb_prod_cat_empty/temp.shape[0]*100, 2), '%)')
print('number of different categories: ', len(cat_prop))
cat_prop[1:]

number of products without this field:  458 ( 41.64 %)
number of different categories:  8


['Composite foods',
 'Fat and sauces',
 'Fish Meat Eggs',
 'Fruits and vegetables',
 'Salty snacks',
 'Sugary snacks',
 'unknown']

In [28]:
#all pnn fields contain a value including unknown
temp[~temp['pnns_groups_1'].isna()].shape

(1100, 11)

In [ ]:
###################################################
# column : pnns_groups_2

In [29]:
cat_prop = values_U_not_nan(temp, 'pnns_groups_2')
nb_prod_cat_empty = df_with_text(temp, 'pnns_groups_2', 'unknown').shape[0]
print( 'number of products without this field: ', nb_prod_cat_empty, '(', round(nb_prod_cat_empty/temp.shape[0]*100, 2), '%)')
print('number of different categories: ', len(cat_prop))
cat_prop[1:]

number of products without this field:  458 ( 41.64 %)
number of different categories:  10


['Fats',
 'Meat',
 'One-dish meals',
 'Salty and fatty products',
 'Soups',
 'Sweets',
 'Unsweetened beverages',
 'Vegetables',
 'unknown']

In [47]:
selection_pnns_groups_2 = [
 #'Fats',
 #'Meat',
 #'One-dish meals',
 'Salty and fatty products',
 #'Soups',
 #'Sweets',
 #'Unsweetened beverages',
 #'Vegetables',
 #'unknown'
]

In [ ]:
###################################################
# column : compared_to_category

In [30]:
cat_prop = values_U_not_nan(temp, 'compared_to_category')
nb_prod_cat_empty = temp['compared_to_category'].isna().sum()
print( 'number of products without this field: ', nb_prod_cat_empty, '(', round(nb_prod_cat_empty/temp.shape[0]*100, 2), '%)')
print('number of different categories: ', len(cat_prop))
cat_prop[1:]

number of products without this field:  444 ( 40.36 %)
number of different categories:  59


['en:arrabbiata-sauces',
 'en:barbecue-sauces',
 'en:bechamel-sauces',
 'en:beverages',
 'en:bolognese-sauces',
 'en:canned-meats',
 'en:cheese-sauces',
 'en:condiments',
 'en:curry-sauces',
 'en:dehydrated-soups',
 'en:fats',
 'en:food',
 'en:fried-tomato-sauces',
 'en:fruit-preserves',
 'en:gazpacho',
 'en:green-pestos',
 'en:harissa-sauces',
 'en:hot-sauces',
 'en:hummus',
 'en:ketchup',
 'en:meals',
 'en:meat-based-sauces',
 'en:mushroom-sauces',
 'en:neapolitan-sauces',
 'en:parmigiano-reggiano',
 'en:pasta-sauces',
 'en:pepper-sauces',
 'en:pestos',
 'en:pimented-sauces',
 'en:pizza-sauces',
 'en:products-without-gluten',
 'en:provencale-sauces',
 'en:puttanesca-sauces',
 'en:red-pestos',
 'en:sauces',
 'en:sauces-for-fishes',
 'en:seasonings',
 'en:tabbouleh',
 'en:teriyaki-sauces',
 'en:tikka-masala-sauce',
 'en:tomato-pastes',
 'en:tomato-sauces',
 'en:tomatoes',
 'en:tomatoes-and-their-products',
 'en:vegetable-sauces',
 'fr:condiments-a-tartiner',
 'fr:pistou',
 'fr:sauce-ap

In [31]:
selection_O_CAT_compared_to_category = [
 'en:arrabbiata-sauces',
 #'en:barbecue-sauces',
 #'en:bechamel-sauces',
 #'en:beverages',
 'en:bolognese-sauces',
 #'en:canned-meats',
 'en:cheese-sauces',
 #'en:condiments',
 'en:curry-sauces',
 #'en:dehydrated-soups',
 #'en:fats',
 #'en:food',
 'en:fried-tomato-sauces',
 #'en:fruit-preserves',
 #'en:gazpacho',
 'en:green-pestos',
 'en:harissa-sauces',
 'en:hot-sauces',
 #'en:hummus',
 'en:ketchup',
 #'en:meals',
 'en:meat-based-sauces',
 'en:mushroom-sauces',
 'en:neapolitan-sauces',
 #'en:parmigiano-reggiano',
 'en:pasta-sauces',
 'en:pepper-sauces',
 'en:pestos',
 'en:pimented-sauces',
 'en:pizza-sauces',
 #'en:products-without-gluten',
 'en:provencale-sauces',
 'en:puttanesca-sauces',
 'en:red-pestos',
 'en:sauces',
 'en:sauces-for-fishes',
 'en:seasonings',
 #'en:tabbouleh',
 'en:teriyaki-sauces',
 'en:tikka-masala-sauce',
 'en:tomato-pastes',
 'en:tomato-sauces',
 'en:tomatoes',
 'en:tomatoes-and-their-products',
 'en:vegetable-sauces',
 'fr:condiments-a-tartiner',
 'fr:pistou',
 'fr:sauce-aperitive',
 'fr:sauces-a-la-creme',
 'fr:sauces-armoricaines',
 'fr:sauces-au-beurre-blanc',
 'fr:sauces-carbonara',
 'fr:sauces-tomates-au-basilic',
 'fr:sauces-tomates-au-parmesan',
 'fr:sauces-tomates-aux-champignons',
 'fr:sauces-tomates-aux-legumes',
 'fr:sauces-tomates-aux-olives',
 'fr:sauces-tomates-pimentees']

In [ ]:
######################################################
#    4) EXTRACT OF OPENFF BASED ON KEY WORDS         #
######################################################

In [48]:
#This time match is defined to include :
# - old products sold in France (first or)
# - AND 
# - criterias based on categories found for NIELSEN products (selection) or products including "sauce" in name and "tomat" in ingredients
#   or pesto in product name
# - this is done to extract retailers brands (MDD)
match = { 
    
    "$and" : [ 
        
        { "$or" : [ 
            
            {"purchase_places" : { '$regex' : "France|Italy", '$options' : "i" }},
            {"countries_tags" : { '$regex' : "en:france|en:italy", '$options' : "i" }},
            
        ]},
        
        
        { "$or" : [ 
    
            {  "categories" : { "$in" : selection_O_CAT_categories } }, 

            { "compared_to_category" : { "$in" : selection_O_CAT_compared_to_category } }, 
            
            { "pnns_groups_2" : { "$in" : selection_pnns_groups_2 } },

            { "category_properties.ciqual_food_name:en" : { "$in" : list(selection_ciqual_food_names_en) } }, 

            { "category_properties.ciqual_food_name:fr" : { "$in" : list(selection_ciqual_food_names_fr) } }, 

            { "$and" : [ 
                {'ingredients' : { "$elemMatch" : { "id" : { '$regex' : "tomat*|pomodoro", '$options' : "i" }}}}, 
                {"product_name_fr" : { '$regex' : "sauce|salsa", '$options' : "i" }}
            ]},
            
            { "$and" : [ 
                {'ingredients' : { "$elemMatch" : { "id" : { '$regex' : "tomat*|pomodoro", '$options' : "i" }}}}, 
                {"product_name_en" : { '$regex' : "sauce|salsa", '$options' : "i" }}
            ]},
            
            {"product_name_fr" : { '$regex' : "pesto", '$options' : "i" }},
            
            {"product_name_en" : { '$regex' : "pesto", '$options' : "i" }}

        ]}
        
]}

In [49]:
c_f = collection.aggregate([
    { '$match' : match } ,
    { '$project' : proj },
    { '$addFields' : addFields }
])

O_data_KW = pd.DataFrame(list(c_f))
O_data_KW.rename(columns={"_id": "EAN13"}, inplace = True)
O_data_KW['brands_tags_str'] = O_data_KW['brands_tags'].astype(str)
O_data_KW.shape

(17648, 25)

In [50]:
# Add two columns based on ciqual_food_name fr and en
import math

O_data_KW['ciqual_food_name:fr'] = ''
O_data_KW['ciqual_food_name:en'] = ''

temp_cfn_fr = []
temp_cfn_en = []

for tc in O_data_KW['category_properties'] :
    
    if isinstance(tc, dict) :
        
        if 'ciqual_food_name:fr' in tc :
            temp_cfn_fr.append(tc['ciqual_food_name:fr'])
        else :
            temp_cfn_fr.append('')
        
        if 'ciqual_food_name:en' in tc :
            temp_cfn_en.append(tc['ciqual_food_name:en'])
        else :
            temp_cfn_en.append('')
    
    else :
        temp_cfn_fr.append('')
        temp_cfn_en.append('')
        
O_data_KW['ciqual_food_name:fr'] = temp_cfn_fr
O_data_KW['ciqual_food_name:en'] = temp_cfn_en

#EXTRACTING ADDITIONAL FEATURES FROM OPEN FF (NUTRIMENTS)
process_O_PRODUCT(O_data_KW)

In [51]:
#Reordering columns
O_data_KW = O_data_KW[[
    'EAN13', 
    'product_name_fr', 'product_name', 
    'brands', 'brands_tags_str', 'brands_tags', 
    'serving_size', 'serving_quantity',
    'countries_hierarchy', 'cities_tags', 'manufacturing_places', 
    'purchase_places', 'countries_tags',
    'categories', 'category_properties', 'pnns_groups_1', 'pnns_groups_2',
    'compared_to_category',
        
    'interface_version_modified',
    
    'ingredients_n',
    'ingredients_text',
    'ingredients', 

    'salt_content', 'max_salt_content', 'min_salt_content',
    'oil_content', 'max_oil_content', 'min_oil_content', 'oil_type',
    
    'nutriments', 
    
    'energy_100g', 'energy_kcal_100g', 'saturated_fat_100g', 'fat_100g',
    'carbohydrates_100g', 'sugars_100g',
    'fruits_vegetables_nuts_estimate_from_ingredients_100g',
    'proteins_100g', 'sodium_100g', 'salt_100g',
    'nutrition_score_fr_100g', 'fiber_100g', 'nova_group_100g']]

In [52]:
#Saving finalized OpenFF export
O_data_KW.to_csv('output_data/O_produits_KW.csv')
O_data_KW.to_excel('output_data/O_produits_KW.xlsx')

In [ ]:
######################################################
#Create O_data_brand

In [ ]:
#check that MDD brands present (auchan, carrefour, casino)

In [35]:
brands_tags_in_KW = O_data_KW[['brands_tags_str', 'EAN13']].groupby(by='brands_tags_str').count().sort_values(by='EAN13', ascending=False)

In [36]:
len(O_data_KW['brands_tags_str'].unique())

1989

In [37]:
brands_tags_in_KW.head(10)

,EAN13
brands_tags_str,
nan,1000
['heinz'],233
[],221
['u'],203
['auchan'],159
['carrefour'],143
['panzani'],139
['sacla'],110
['casino'],89


In [39]:
brands_tags_in_KW.to_csv('output_data/O_brands_tags_in_KW')

In [40]:
brands_tags_in_EAN13 = O_data_EAN13[['brands_tags_str', 'EAN13']].groupby(by='brands_tags_str').count().sort_values(by='EAN13', ascending=False)

In [41]:
print("number of brands in NIELSEN : ", brands_tags_in_EAN13.shape[0])
print("number of brands in KW : ", brands_tags_in_KW.shape[0])
print("number of brands in both : ", brands_tags_in_KW.isin(brands_tags_in_EAN13)['EAN13'].sum())

number of brands in NIELSEN :  202
number of brands in KW :  1989
number of brands in both :  69
